In [1]:
!pip install --upgrade pip
!pip install gcsfs
!pip install spacy

Requirement already up-to-date: pip in /opt/conda/lib/python3.7/site-packages (20.0.2)
     |████████████████████████████████| 10.6 MB 2.6 MB/s eta 0:00:01
     |████████████████████████████████| 3.7 MB 65.1 MB/s eta 0:00:01
     |████████████████████████████████| 2.2 MB 74.3 MB/s eta 0:00:01
     |████████████████████████████████| 185 kB 90.2 MB/s eta 0:00:01
     |████████████████████████████████| 118 kB 100.5 MB/s eta 0:00:01


In [2]:
%%time
import os, sys, time, json, re, string, datetime 
import multiprocessing as mp
import pandas as pd
import numpy as np
import spacy
from google.cloud import storage

import tensorflow as tf
import keras

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional, MaxPooling1D, CuDNNLSTM, Conv1D, CuDNNGRU

import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 

config = tf.ConfigProto( device_count = {'GPU': 1 , 'CPU': 16} ) 
sess = tf.Session(config=config) 
keras.backend.set_session(sess)

nltk.download('stopwords')
stop_words = set(stopwords.words('english')) 

Using TensorFlow backend.


CPU times: user 2.33 s, sys: 1.31 s, total: 3.64 s
Wall time: 12 s


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jupyter/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
%%time
df_train = pd.read_csv('gs://dataproc-6ca41800-27b4-47d5-abee-55c011dfa389-asia-southeast1/data/kaggle/train.csv')
df_train.info()
df_train['Outcome'].value_counts()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44183 entries, 0 to 44182
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Comment  44183 non-null  object
 1   Outcome  44183 non-null  int64 
 2   Id       44183 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.0+ MB
CPU times: user 312 ms, sys: 92 ms, total: 404 ms
Wall time: 2.72 s


1    24992
0    19191
Name: Outcome, dtype: int64

In [10]:
%%time
max_words = 1000
tk = Tokenizer(lower = True)
tk.fit_on_texts(df_train.Comment)
df_train_seq = tk.texts_to_sequences(df_train.Comment)
df_train_pad = pad_sequences(df_train_seq, maxlen=max_words, padding='post')

vocabulary_size = len(tk.word_counts.keys())+1
X_train, X_test, y_train, y_test = train_test_split(df_train_pad, df_train.Outcome, test_size = 0.2, random_state = 1)
X_train_data, X_validate_data, y_train_data, y_validate_data = train_test_split(X_train, y_train, test_size = 0.2, random_state = 1)

CPU times: user 6.6 s, sys: 140 ms, total: 6.74 s
Wall time: 6.73 s


In [12]:
model = Sequential()
model.add(Embedding(vocabulary_size, 150, input_length=max_words))
model.add(Bidirectional(CuDNNLSTM(256)))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train_data, y_train_data, validation_data=(X_validate_data, y_validate_data), batch_size=64, epochs=5)

Train on 28276 samples, validate on 7070 samples
Epoch 1/5
28276/28276 [==============================] - 106s 4ms/step - loss: 0.6199 - accuracy: 0.6602 - val_loss: 0.5972 - val_accuracy: 0.6908
Epoch 2/5
28276/28276 [==============================] - 105s 4ms/step - loss: 0.5071 - accuracy: 0.7589 - val_loss: 0.6132 - val_accuracy: 0.6809
Epoch 3/5
28276/28276 [==============================] - 105s 4ms/step - loss: 0.4151 - accuracy: 0.8173 - val_loss: 0.6686 - val_accuracy: 0.6818
Epoch 4/5
28276/28276 [==============================] - 105s 4ms/step - loss: 0.3024 - accuracy: 0.8757 - val_loss: 0.6786 - val_accuracy: 0.6806
Epoch 5/5
28276/28276 [==============================] - 105s 4ms/step - loss: 0.2127 - accuracy: 0.9164 - val_loss: 0.8717 - val_accuracy: 0.6635


In [13]:
score = model.evaluate(X_test, y_test, verbose = 0)
print('Test accuracy:', score[1])

Test accuracy: 0.6611972451210022
